## API Example

In [1]:
!pip install --upgrade pip
!pip install transformers accelerate datasets peft bitsandbytes torch matplotlib
!pip install auto-gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# from peft import PeftModel
import torch
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from huggingface_hub import login

In [3]:
login()

In [4]:
# 1. 모델 불러오기 (4bit quantized 단독 모델)
model_id = "capston-team-5/finma-7b-4bit-quantized"

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_cfg,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:609: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

In [5]:
# 2. 추론 함수 정의
def generate_response(prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [6]:
def run_prediction(ticker="005930.KS", horizon_days=1):
    assert horizon_days in [1, 7, 30], "Only 1-day, 7-day, or 30-day forecasts are supported."

    # 기준 날짜
    today = datetime.today()
    predict_date = today + timedelta(days=horizon_days)

    # 날짜 설정
    if horizon_days == 1:
        interval = "1d"
        fetch_days = 15
    elif horizon_days == 7:
        interval = "1wk"
        fetch_days = 70  # 10주 이상 확보
    elif horizon_days == 30:
        interval = "1mo"
        fetch_days = 300  # 10개월 이상 확보

    start_date = today - timedelta(days=fetch_days)
    end_date = today

    # 데이터 수집
    data = yf.download(
        ticker,
        start=start_date.strftime("%Y-%m-%d"),
        end=(end_date + timedelta(days=1)).strftime("%Y-%m-%d"),
        interval=interval
    )

    # 전처리
    data = data[["Open", "High", "Low", "Close", "Volume"]]
    data.reset_index(inplace=True)
    data["Date"] = data["Date"].dt.strftime("%Y-%m-%d")
    data["Change"] = data["Close"].pct_change().fillna(0)

    # 최근 10영업일 추출
    last_10 = data.tail(10)

    # 프롬프트 생성
    def extract(val):
        return val.iloc[0] if isinstance(val, pd.Series) else val

    context = "\n".join([
        f"{extract(row['Date'])} ({interval}), {int(extract(row['Open']))}, {int(extract(row['High']))}, "
        f"{int(extract(row['Low']))}, {int(extract(row['Close']))}, {int(extract(row['Volume']))}, "
        f"{extract(row['Change']):.16f}"
        for _, row in last_10.iterrows()
    ])

    predict_date_str = predict_date.strftime("%Y-%m-%d")
    prompt = (
        f"Assess the data to estimate how the closing price of {ticker} will change on {predict_date_str}.\n"
        "Respond with one of the following levels based on the rate of change:\n"
        "Strong Rise (≥ 5%), Rise (2%–4.99%), Slight Rise (0%–1.99%), "
        "Slight Fall (–1.99% to 0%), Fall (–4.99% to –2%), or Strong Fall (≤ –5%).\n\n"
        "Context: date, open, high, low, close, volume, change.\n"
        f"{context}\n"
        "Answer:"
    )

    # 추론 실행
    response = generate_response(prompt)
    answer_prefix = "Answer:"
    if answer_prefix in response:
        final_output = response.split(answer_prefix, 1)[-1].strip()
    else:
        final_output = response.strip()

    # 출력
    print(f"\n[예측 대상 날짜: {predict_date_str}]")
    print("[Model Output]")
    print(final_output)

    print()
    print(prompt)

    return final_output, prompt

In [7]:
run_prediction(ticker="005930.KS", horizon_days=1)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed



[예측 대상 날짜: 2025-05-16]
[Model Output]
Rise

Assess the data to estimate how the closing price of 005930.KS will change on 2025-05-16.
Respond with one of the following levels based on the rate of change:
Strong Rise (≥ 5%), Rise (2%–4.99%), Slight Rise (0%–1.99%), Slight Fall (–1.99% to 0%), Fall (–4.99% to –2%), or Strong Fall (≤ –5%).

Context: date, open, high, low, close, volume, change.
2025-04-30 (1d), 55900, 56000, 55300, 55500, 13014189, 0.0000000000000000
2025-05-02 (1d), 55000, 55500, 54200, 54300, 22454204, -0.0216216216216216
2025-05-07 (1d), 54500, 55000, 54300, 54600, 17136991, 0.0055248618784531
2025-05-08 (1d), 55100, 55500, 54500, 54600, 16320532, 0.0000000000000000
2025-05-09 (1d), 54700, 55000, 54400, 54800, 7814322, 0.0036630036630036
2025-05-12 (1d), 55200, 57600, 55000, 57600, 15414702, 0.0510948905109490
2025-05-13 (1d), 57700, 58200, 56800, 56900, 16842801, -0.0121527777777778
2025-05-14 (1d), 57500, 57900, 57000, 57400, 12468089, 0.0087873462214412
2025-05-15 

('Rise',
 'Assess the data to estimate how the closing price of 005930.KS will change on 2025-05-16.\nRespond with one of the following levels based on the rate of change:\nStrong Rise (≥ 5%), Rise (2%–4.99%), Slight Rise (0%–1.99%), Slight Fall (–1.99% to 0%), Fall (–4.99% to –2%), or Strong Fall (≤ –5%).\n\nContext: date, open, high, low, close, volume, change.\n2025-04-30 (1d), 55900, 56000, 55300, 55500, 13014189, 0.0000000000000000\n2025-05-02 (1d), 55000, 55500, 54200, 54300, 22454204, -0.0216216216216216\n2025-05-07 (1d), 54500, 55000, 54300, 54600, 17136991, 0.0055248618784531\n2025-05-08 (1d), 55100, 55500, 54500, 54600, 16320532, 0.0000000000000000\n2025-05-09 (1d), 54700, 55000, 54400, 54800, 7814322, 0.0036630036630036\n2025-05-12 (1d), 55200, 57600, 55000, 57600, 15414702, 0.0510948905109490\n2025-05-13 (1d), 57700, 58200, 56800, 56900, 16842801, -0.0121527777777778\n2025-05-14 (1d), 57500, 57900, 57000, 57400, 12468089, 0.0087873462214412\n2025-05-15 (1d), 57400, 58600, 5

## Deploy

In [8]:
!pip install fastapi uvicorn nest-asyncio pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [fastapi]


In [9]:
!ngrok config add-authtoken 2x25AEVCRyeF55jkzzZOGxVLnFv_5WXMn3DcsdT71GEz8a9sP

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
import nest_asyncio
from fastapi import FastAPI, Query
from pyngrok import ngrok
from threading import Thread
import uvicorn
from datetime import datetime, timedelta

# 2. 필수 초기화
nest_asyncio.apply()
app = FastAPI()

# 3. 예제 라우터 (GET)
@app.get("/predict")
def predict(ticker: str = Query(...), horizon_days: int = Query(...)):
    result, prompt = run_prediction(ticker, horizon_days)
    return {
        "ticker": ticker,
        "horizon_days": horizon_days,
        "prediction_date": (datetime.today() + timedelta(days=horizon_days)).strftime("%Y-%m-%d"),
        "prompt": prompt,
        "prediction_result": result
    }

# 4. 비동기 서버 실행 함수
port = 7861
def start_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=port)

# 5. 백그라운드로 FastAPI 실행
Thread(target=start_uvicorn).start()

# 6. ngrok 연결
public_url = ngrok.connect(port)
print(f"🚀 FastAPI endpoint: {public_url}/predict?ticker=AAPL&horizon_days=7")

INFO:     Started server process [292]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7861 (Press CTRL+C to quit)


🚀 FastAPI endpoint: NgrokTunnel: "https://3d98-34-168-211-219.ngrok-free.app" -> "http://localhost:7861"/predict?ticker=AAPL&horizon_days=7


In [11]:
import time
_time = 0
while(True):
    time.sleep(300)
    _time += 300
    print("Running Time:", _time, "sec")
    continue


INFO:     203.252.33.5:0 - "GET /predict HTTP/1.1" 422 Unprocessable Entity
INFO:     203.252.33.5:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     203.252.33.5:0 - "GET /predict?ticker=AAPL&horizon=7 HTTP/1.1" 422 Unprocessable Entity


[*********************100%***********************]  1 of 1 completed



[예측 대상 날짜: 2025-05-22]
[Model Output]
Rise

Assess the data to estimate how the closing price of AAPL will change on 2025-05-22.
Respond with one of the following levels based on the rate of change:
Strong Rise (≥ 5%), Rise (2%–4.99%), Slight Rise (0%–1.99%), Slight Fall (–1.99% to 0%), Fall (–4.99% to –2%), or Strong Fall (≤ –5%).

Context: date, open, high, low, close, volume, change.
2025-03-10 (1wk), 235, 235, 208, 213, 332232000, -0.1069979222444847
2025-03-17 (1wk), 213, 218, 209, 217, 287881900, 0.0223897473963637
2025-03-24 (1wk), 220, 224, 217, 217, 190172600, -0.0016951787609648
2025-03-31 (1wk), 216, 225, 187, 188, 366947800, -0.1354749175267389
2025-04-07 (1wk), 176, 200, 168, 197, 675037600, 0.0518631568634067
2025-04-14 (1wk), 211, 212, 192, 196, 263763500, -0.0059045639080602
2025-04-21 (1wk), 193, 209, 189, 209, 238181400, 0.0624428780577666
2025-04-28 (1wk), 209, 214, 201, 205, 286233500, -0.0187786568732838
2025-05-05 (1wk), 202, 203, 192, 198, 275704500, -0.03321157

[*********************100%***********************]  1 of 1 completed



[예측 대상 날짜: 2025-05-22]
[Model Output]
Rise

Assess the data to estimate how the closing price of AAPL will change on 2025-05-22.
Respond with one of the following levels based on the rate of change:
Strong Rise (≥ 5%), Rise (2%–4.99%), Slight Rise (0%–1.99%), Slight Fall (–1.99% to 0%), Fall (–4.99% to –2%), or Strong Fall (≤ –5%).

Context: date, open, high, low, close, volume, change.
2025-03-10 (1wk), 235, 235, 208, 213, 332232000, -0.1069979222444847
2025-03-17 (1wk), 213, 218, 209, 217, 287881900, 0.0223897473963637
2025-03-24 (1wk), 220, 224, 217, 217, 190172600, -0.0016951787609648
2025-03-31 (1wk), 216, 225, 187, 188, 366947800, -0.1354749175267389
2025-04-07 (1wk), 176, 200, 168, 197, 675037600, 0.0518631568634067
2025-04-14 (1wk), 211, 212, 192, 196, 263763500, -0.0059045639080602
2025-04-21 (1wk), 193, 209, 189, 209, 238181400, 0.0624428780577666
2025-04-28 (1wk), 209, 214, 201, 205, 286233500, -0.0187786568732838
2025-05-05 (1wk), 202, 203, 192, 198, 275704500, -0.03321157

...

[*********************100%***********************]  1 of 1 completed



[예측 대상 날짜: 2025-06-14]
[Model Output]
Rise

Assess the data to estimate how the closing price of 018260.KS will change on 2025-06-14.
Respond with one of the following levels based on the rate of change:
Strong Rise (≥ 5%), Rise (2%–4.99%), Slight Rise (0%–1.99%), Slight Fall (–1.99% to 0%), Fall (–4.99% to –2%), or Strong Fall (≤ –5%).

Context: date, open, high, low, close, volume, change.
2024-08-01 (1mo), 144387, 150643, 130896, 147320, 2533655, 0.0000000000000000
2024-09-01 (1mo), 147320, 162276, 144680, 151230, 3066119, 0.0265427669056377
2024-10-01 (1mo), 150350, 152305, 138424, 139890, 2581302, -0.0749838305399256
2024-11-01 (1mo), 140281, 150546, 130310, 143409, 3563955, 0.0251572910103450
2024-12-01 (1mo), 145755, 147808, 122587, 124933, 3534688, -0.1288344019496582
2025-01-01 (1mo), 127500, 132000, 113100, 120700, 3584825, -0.0338871148220812
2025-02-01 (1mo), 119200, 132300, 118300, 123300, 2867872, 0.0215410107705054
2025-03-01 (1mo), 122900, 124200, 116000, 116200, 22926

[*********************100%***********************]  1 of 1 completed



[예측 대상 날짜: 2025-05-16]
[Model Output]
Rise

Assess the data to estimate how the closing price of 000660.KS will change on 2025-05-16.
Respond with one of the following levels based on the rate of change:
Strong Rise (≥ 5%), Rise (2%–4.99%), Slight Rise (0%–1.99%), Slight Fall (–1.99% to 0%), Fall (–4.99% to –2%), or Strong Fall (≤ –5%).

Context: date, open, high, low, close, volume, change.
2025-04-30 (1d), 179100, 180000, 176700, 177500, 2354473, 0.0000000000000000
2025-05-02 (1d), 179400, 186200, 178400, 186000, 3886874, 0.0478873239436619
2025-05-07 (1d), 187700, 191500, 185900, 190800, 3830097, 0.0258064516129033
2025-05-08 (1d), 196000, 196000, 189400, 190300, 5125330, -0.0026205450733753
2025-05-09 (1d), 192000, 192800, 186800, 190100, 2539822, -0.0010509721492380
2025-05-12 (1d), 193600, 195200, 193500, 195000, 1624884, 0.0257759074171489
2025-05-13 (1d), 200500, 201500, 197300, 198500, 3385009, 0.0179487179487179
2025-05-14 (1d), 207000, 207000, 204000, 206000, 3333511, 0.037

KeyboardInterrupt: 